In [58]:
import os
import shutil
import datetime
import subprocess
import re
import multiprocessing
import time
import json
import datetime
import time
import boto3

In [7]:

SIMULATION_ROOT_DIR = "simulation"
SIMULATION_EXEC = os.path.join(SIMULATION_ROOT_DIR, "bin")
SIMULATION_LIBS = os.path.join(SIMULATION_ROOT_DIR, "libs")
SIMULATION_RUN_DIRS= os.path.join(SIMULATION_ROOT_DIR, "runs")


os.environ["PATH"] += os.pathsep + os.path.join(os.getcwd(), SIMULATION_EXEC)
os.environ["PATH"]

'/home/ubuntu/anaconda3/envs/lammps/bin:/home/ubuntu/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/ubuntu/CTP/simulation/bin'

In [8]:
def get_curr_time():
    return datetime.datetime.now().strftime("%d-%h-%y | %H:%M")

def _load_seq_from_file(file_name):
    with open(file_name) as f:
        seq = f.readlines()
    return "".join([l.strip() for l in seq])

def create_run_dir_name(gene_name):
    return gene_name + "_" + datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")

In [9]:
def gen_sim_data(folder_name):
    seq = _load_seq_from_file(os.path.join(folder_name, "seq.txt"))
    seq_name = folder_name.split(os.sep)[-1]
    dirs = next(os.walk(os.path.join(folder_name, 'directCoexistence')))[1]
    dirs = [dir[4:] for dir in dirs]
    return {"gene_name": seq_name, "seq": seq, "temps": dirs}

def gen_sim_data_done(folder_name):
    seq = _load_seq_from_file(os.path.join(folder_name, "seq.txt"))
    seq_name = folder_name.split(os.sep)[-1]
    dirs = next(os.walk(folder_name))[1]
    dirs = [dir[4:] for dir in dirs]
    return {"gene_name": seq_name, "seq": seq, "temps": dirs}

def read_sim_data(seq_file='simulation/simulation_to_run.json'):
    with open(seq_file) as f:
        return json.load(f)

def run_sample_lmp(temp_dir_name, CPUs_per_process):
    f = open(os.path.join(temp_dir_name, "sample.log"), "w+")
    CPUs_per_process_arg = str(CPUs_per_process)
    subprocess.run(["mpiexec", "-np", CPUs_per_process_arg, "lmp", "-in", "sample.lmp"], stdout=f, cwd=temp_dir_name)
    f.close()

In [10]:
def get_completed_simulations():
    sim_dirs = next(os.walk(os.path.join("simulation", "runs")))[1]
    comp_sims = ["_".join(dir_name.split("_")[:-6]) for dir_name in sim_dirs if dir_name!='']
    return [sim for sim in comp_sims if sim]
simulations_completed = get_completed_simulations()
# simulations_completed

In [45]:
def get_start_temp(seq):
    return 300

In [20]:
seq_to_run = read_sim_data()
seq_to_run = {seq: seq_to_run[seq]["seq"] for seq in seq_to_run}
for seq in simulations_completed:
    seq_to_run.pop(seq, None)

In [49]:
seq_size_limit = 50
seq_to_run_test = {seq: seq_to_run[seq][:seq_size_limit] for seq in seq_to_run}

In [50]:
seq_to_run_test = [(seq_name,
                    seq_to_run_test[seq_name],
                    get_start_temp(seq_to_run_test[seq_name])) for seq_name in seq_to_run_test]

In [59]:
def get_dynamodb():
    return boto3.resource('dynamodb',
                          region_name='us-east-2', 
                          aws_access_key_id=None,
                          aws_secret_access_key=None)

dynamodb = get_dynamodb()

slots_table = dynamodb.Table('slots')
simulation_table = dynamodb.Table('simulations')

In [62]:
def add_slots(slots_table):
    CPUs_pp = 16
    CPUs_total = 96
    for i in range(CPUs_total // CPUs_pp):
        slots_table.put_item(Item={'id': i, 
                             'empty': True,
                             'extra_info': ""})

In [63]:
add_slots(slots_table)

NoCredentialsError: Unable to locate credentials

In [26]:
def get_free_slot():
    response = table.scan()

In [30]:
def get_new_running_simulation():
    pass

In [32]:
def update_running_simualtions():
    pass

In [31]:
def get_temp_for_sim(sim_id):
    pass

In [41]:
def update_seq_to_run(seq_to_run):
    update_running_simualtions()
    return seq_to_run + get_new_running_simulation()
    

In [42]:
def run_sumilation(seq_data, slot):
    pass

In [ ]:
free_slot_wait_period = 5
loop_wait_preiod = 60
while seq_to_run:
    
    free_slots = get_free_slots()
    
    if free_slots:
        time.sleep(free_slot_wait_period)
        seq_to_run = update_seq_to_run(seq_to_run)
    
    while(free_slots):
        for slot in free_slots:
            run_sumilation(seq_to_run.pop(), slot)

    
    time.sleep(loop_wait_preiod)        
                
                  

In [38]:
z = [1,2,3]

In [ ]:
while(z):
    print(z)
    z.pop()
    z.append(1)

In [29]:
time.sleep(5)